## Write to CSV

In [1]:
import os
import csv
import pandas as pd
from pandasql import sqldf
from collections import defaultdict

range_names = ["00 LJ vs BB 3Bet", "01 LJ vs BB Call", "02 LJ vs HJ 3Bet", "03 LJ vs CO 3Bet", "04 LJ vs BN 3Bet", "05 LJ vs SB 3Bet", "06 LJ vs HJ Call", "07 LJ vs CO Call", "08 LJ vs BN Call", "09 LJ vs SB Call", "10 HJ vs BB 3Bet", "11 HJ vs BB Call", "12 HJ vs CO 3Bet", "13 HJ vs BN 3Bet", "14 HJ vs SB 3Bet", "15 HJ vs CO Call", "16 HJ vs BN Call", "17 HJ vs SB Call", "18 CO vs BB 3Bet", "19 CO vs BB Call", "20 CO vs BN 3Bet", "21 CO vs SB 3Bet", "22 CO vs BN Call", "23 CO vs SB Call", "24 BN vs BB 3Bet", "25 BN vs BB Call", "26 BN vs SB 3Bet", "27 SB vs BB 3Bet", "28 SB vs BB Call", "29 BB 3Bet vs LJ", "30 BB Call vs LJ", "31 HJ 3Bet vs LJ", "32 CO 3Bet vs LJ", "33 BN 3Bet vs LJ", "34 SB 3Bet vs LJ", "35 HJ Call vs LJ", "36 CO Call vs LJ", "37 BN Call vs LJ", "38 SB Call vs LJ", "39 BB 3Bet vs HJ", "40 BB Call vs HJ", "41 CO 3Bet vs HJ", "42 BN 3Bet vs HJ", "43 SB 3Bet vs HJ", "44 CO Call vs HJ", "45 BN Call vs HJ", "46 SB Call vs HJ", "47 BB 3Bet vs CO", "48 BB Call vs CO", "49 BN 3Bet vs CO", "50 SB 3Bet vs CO", "51 BN Call vs CO", "52 SB Call vs CO", "53 BB 3Bet vs BN", "54 BB Call vs BN", "55 SB 3Bet vs BN", "56 BB 3Bet vs SB", "57 BB Call vs SB",]
board_type = "two-tone"   # two-tone, rainbow, monotone


In [2]:

for range_name in range_names:    
    df2 = None
    dfs = []
    for i in range(500):
        try:
            dfs.append(pd.read_csv("../reports/{}/{}/{}.csv".format(range_name, board_type, str(i).zfill(3))).drop(["Unnamed: 0"], axis=1))
        except:
            pass
    df = pd.concat(dfs, axis=0)
    df2 = sqldf("""select * from df a
            INNER JOIN (select flop, board_type, max(profit) AS max_profit from df group by 1,2) b on
            b.flop = a.flop and b.board_type = a.board_type and a.profit = b.max_profit
           order by 1 desc"""
          ,globals())
    try:
        os.mkdir("../reports/condensed_results/{}".format(board_type))
    except:
        pass
    df2.to_csv("../reports/condensed_results/{}/{}.csv".format(board_type, range_name))
    df2.head()




ValueError: No objects to concatenate

## Check Average Profit Per Situation

Tighten up ranges of the negative values proportional to their negative value

In [3]:
averages = []
try:
    for range_name in range_names:
        df = pd.read_csv("../reports/condensed_results/two-tone/{}.csv".format(range_name))
        df = df[['profit']]
        averages.append(df.mean()['profit'])
except:
    pass
averages
    

[3.779741652415367,
 1.6097499101922377,
 -5.126884541070893,
 -5.126749803336997,
 -3.0551769819976236,
 4.772106450011192,
 0.9686143821104387,
 1.0032170903549864,
 1.2137343183527276,
 1.5610827597855288,
 3.6304850709516803,
 1.7900667650582234,
 -1.4216406386266354,
 0.3980184934021703,
 4.988090433115355,
 0.8450216802810219,
 1.0445137050191602,
 1.7968489762830016,
 5.227868571142115,
 2.0733732111219627,
 1.6136971409207848,
 5.730962373230267,
 1.1110017149966913,
 2.1655882609683648,
 5.723281873378416,
 2.049173730688076,
 6.470934612120427,
 0.9446206598989962,
 0.9162923076168576,
 6.589008132308526,
 1.0857188004216343,
 6.154804199200054,
 6.154875239095216,
 4.772775768687437,
 5.521834091373802,
 2.736075188250723,
 2.7473178364449646,
 2.826855150793056,
 1.433243255425964,
 5.905693246979601,
 1.135061360353408,
 4.95351122535019,
 4.736513138070179,
 4.513752464526848,
 3.0924984841025296,
 2.787476705363098,
 1.3441549340107355,
 3.9552173530675154,
 0.9808214862

## Write to Formatted CSV

In [4]:

for range_name in range_names:
    df = pd.read_csv("../reports/condensed_results/two-tone/{}.csv".format(range_name))
    df["rank1"] = df["flop"].apply(lambda x: x.replace("[","").replace("]","").split(",")[0]).astype(int)
    df["rank2"] = df["flop"].apply(lambda x: x.replace("[","").replace("]","").split(",")[1]).astype(int)
    df["rank3"] = df["flop"].apply(lambda x: x.replace("[","").replace("]","").split(",")[2]).astype(int)
    df.sort_values(by=["rank1", "rank2", "rank3"], inplace=True, ascending=False)
    df = df[['rank1', 'rank2', 'rank3', '0b']]
    a = defaultdict(str)
    for index, row in df.iterrows():
        a[(row['rank1'], row['rank2'], row['rank3'])] = row['0b']
    with open('../reports/final_results/{}/{}.csv'.format(board_type, range_name), mode='w') as fout:
        csv_writer = csv.writer(fout, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        csv_writer.writerow(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', range_name, '','','','','','','','','','','','','','','','','','','','','','','','',''])
        csv_writer.writerow(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '','','','','','','','','','','','','','','','','','','','','','','','',''])
        csv_writer.writerow(['', 'A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2', '', '', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2', '', '', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2'])
        csv_writer.writerow(['A', a[(14,14,14)], a[(14,14,13)], a[(14,14,12)], a[(14,14,11)], a[(14,14,10)], a[(14,14,9)], a[(14,14,8)], a[(14,14,7)], a[(14,14,6)], a[(14,14,5)], a[(14,14,4)], a[(14,14,3)], a[(14,14,2)], '', 'K', a[(13,13,13)], a[(13,13,12)], a[(13,13,11)], a[(13,13,10)], a[(13,13,9)], a[(13,13,8)], a[(13,13,7)], a[(13,13,6)], a[(13,13,5)], a[(13,13,4)], a[(13,13,3)], a[(13,13,2)], '', 'Q', a[(12,12,12)], a[(12,12,11)], a[(12,12,10)], a[(12,12,9)], a[(12,12,8)], a[(12,12,7)], a[(12,12,6)], a[(12,12,5)], a[(12,12,4)], a[(12,12,3)], a[(12,12,2)], a[(12,12,1)], a[(12,12,0)]])
        csv_writer.writerow(['K', a[(14,13,14)], a[(14,13,13)], a[(14,13,12)], a[(14,13,11)], a[(14,13,10)], a[(14,13,9)], a[(14,13,8)], a[(14,13,7)], a[(14,13,6)], a[(14,13,5)], a[(14,13,4)], a[(14,13,3)], a[(14,13,2)], '', 'Q', a[(13,12,13)], a[(13,12,12)], a[(13,12,11)], a[(13,12,10)], a[(13,12,9)], a[(13,12,8)], a[(13,12,7)], a[(13,12,6)], a[(13,12,5)], a[(13,12,4)], a[(13,12,3)], a[(13,12,2)], '', 'J', a[(12,11,12)], a[(12,11,11)], a[(12,11,10)], a[(12,11,9)], a[(12,11,8)], a[(12,11,7)], a[(12,11,6)], a[(12,11,5)], a[(12,11,4)], a[(12,11,3)], a[(12,11,2)], a[(12,11,1)], a[(12,11,0)]])
        csv_writer.writerow(['Q', a[(14,12,14)], a[(14,12,13)], a[(14,12,12)], a[(14,12,11)], a[(14,12,10)], a[(14,12,9)], a[(14,12,8)], a[(14,12,7)], a[(14,12,6)], a[(14,12,5)], a[(14,12,4)], a[(14,12,3)], a[(14,12,2)], '', 'J', a[(13,11,13)], a[(13,11,12)], a[(13,11,11)], a[(13,11,10)], a[(13,11,9)], a[(13,11,8)], a[(13,11,7)], a[(13,11,6)], a[(13,11,5)], a[(13,11,4)], a[(13,11,3)], a[(13,11,2)], '', 'T', a[(12,10,12)], a[(12,10,11)], a[(12,10,10)], a[(12,10,9)], a[(12,10,8)], a[(12,10,7)], a[(12,10,6)], a[(12,10,5)], a[(12,10,4)], a[(12,10,3)], a[(12,10,2)], a[(12,10,1)], a[(12,10,0)]])
        csv_writer.writerow(['J', a[(14,11,14)], a[(14,11,13)], a[(14,11,12)], a[(14,11,11)], a[(14,11,10)], a[(14,11,9)], a[(14,11,8)], a[(14,11,7)], a[(14,11,6)], a[(14,11,5)], a[(14,11,4)], a[(14,11,3)], a[(14,11,2)], '', 'T', a[(13,10,13)], a[(13,10,12)], a[(13,10,11)], a[(13,10,10)], a[(13,10,9)], a[(13,10,8)], a[(13,10,7)], a[(13,10,6)], a[(13,10,5)], a[(13,10,4)], a[(13,10,3)], a[(13,10,2)], '', '9', a[(12,9,12)], a[(12,9,11)], a[(12,9,10)], a[(12,9,9)], a[(12,9,8)], a[(12,9,7)], a[(12,9,6)], a[(12,9,5)], a[(12,9,4)], a[(12,9,3)], a[(12,9,2)], a[(12,9,1)], a[(12,9,0)]])
        csv_writer.writerow(['T', a[(14,10,14)], a[(14,10,13)], a[(14,10,12)], a[(14,10,11)], a[(14,10,10)], a[(14,10,9)], a[(14,10,8)], a[(14,10,7)], a[(14,10,6)], a[(14,10,5)], a[(14,10,4)], a[(14,10,3)], a[(14,10,2)], '', '9', a[(13,9,13)], a[(13,9,12)], a[(13,9,11)], a[(13,9,10)], a[(13,9,9)], a[(13,9,8)], a[(13,9,7)], a[(13,9,6)], a[(13,9,5)], a[(13,9,4)], a[(13,9,3)], a[(13,9,2)], '', '8', a[(12,8,12)], a[(12,8,11)], a[(12,8,10)], a[(12,8,9)], a[(12,8,8)], a[(12,8,7)], a[(12,8,6)], a[(12,8,5)], a[(12,8,4)], a[(12,8,3)], a[(12,8,2)], a[(12,8,1)], a[(12,8,0)]])
        csv_writer.writerow(['9', a[(14,9,14)], a[(14,9,13)], a[(14,9,12)], a[(14,9,11)], a[(14,9,10)], a[(14,9,9)], a[(14,9,8)], a[(14,9,7)], a[(14,9,6)], a[(14,9,5)], a[(14,9,4)], a[(14,9,3)], a[(14,9,2)], '', '8', a[(13,8,13)], a[(13,8,12)], a[(13,8,11)], a[(13,8,10)], a[(13,8,9)], a[(13,8,8)], a[(13,8,7)], a[(13,8,6)], a[(13,8,5)], a[(13,8,4)], a[(13,8,3)], a[(13,8,2)], '', '7', a[(12,7,12)], a[(12,7,11)], a[(12,7,10)], a[(12,7,9)], a[(12,7,8)], a[(12,7,7)], a[(12,7,6)], a[(12,7,5)], a[(12,7,4)], a[(12,7,3)], a[(12,7,2)], a[(12,7,1)], a[(12,7,0)]])
        csv_writer.writerow(['8', a[(14,8,14)], a[(14,8,13)], a[(14,8,12)], a[(14,8,11)], a[(14,8,10)], a[(14,8,9)], a[(14,8,8)], a[(14,8,7)], a[(14,8,6)], a[(14,8,5)], a[(14,8,4)], a[(14,8,3)], a[(14,8,2)], '', '7', a[(13,7,13)], a[(13,7,12)], a[(13,7,11)], a[(13,7,10)], a[(13,7,9)], a[(13,7,8)], a[(13,7,7)], a[(13,7,6)], a[(13,7,5)], a[(13,7,4)], a[(13,7,3)], a[(13,7,2)], '', '6', a[(12,6,12)], a[(12,6,11)], a[(12,6,10)], a[(12,6,9)], a[(12,6,8)], a[(12,6,7)], a[(12,6,6)], a[(12,6,5)], a[(12,6,4)], a[(12,6,3)], a[(12,6,2)], a[(12,6,1)], a[(12,6,0)]])
        csv_writer.writerow(['7', a[(14,7,14)], a[(14,7,13)], a[(14,7,12)], a[(14,7,11)], a[(14,7,10)], a[(14,7,9)], a[(14,7,8)], a[(14,7,7)], a[(14,7,6)], a[(14,7,5)], a[(14,7,4)], a[(14,7,3)], a[(14,7,2)], '', '6', a[(13,6,13)], a[(13,6,12)], a[(13,6,11)], a[(13,6,10)], a[(13,6,9)], a[(13,6,8)], a[(13,6,7)], a[(13,6,6)], a[(13,6,5)], a[(13,6,4)], a[(13,6,3)], a[(13,6,2)], '', '5', a[(12,5,12)], a[(12,5,11)], a[(12,5,10)], a[(12,5,9)], a[(12,5,8)], a[(12,5,7)], a[(12,5,6)], a[(12,5,5)], a[(12,5,4)], a[(12,5,3)], a[(12,5,2)], a[(12,5,1)], a[(12,5,0)]])
        csv_writer.writerow(['6', a[(14,6,14)], a[(14,6,13)], a[(14,6,12)], a[(14,6,11)], a[(14,6,10)], a[(14,6,9)], a[(14,6,8)], a[(14,6,7)], a[(14,6,6)], a[(14,6,5)], a[(14,6,4)], a[(14,6,3)], a[(14,6,2)], '', '5', a[(13,5,13)], a[(13,5,12)], a[(13,5,11)], a[(13,5,10)], a[(13,5,9)], a[(13,5,8)], a[(13,5,7)], a[(13,5,6)], a[(13,5,5)], a[(13,5,4)], a[(13,5,3)], a[(13,5,2)], '', '4', a[(12,4,12)], a[(12,4,11)], a[(12,4,10)], a[(12,4,9)], a[(12,4,8)], a[(12,4,7)], a[(12,4,6)], a[(12,4,5)], a[(12,4,4)], a[(12,4,3)], a[(12,4,2)], a[(12,4,1)], a[(12,4,0)]])
        csv_writer.writerow(['5', a[(14,5,14)], a[(14,5,13)], a[(14,5,12)], a[(14,5,11)], a[(14,5,10)], a[(14,5,9)], a[(14,5,8)], a[(14,5,7)], a[(14,5,6)], a[(14,5,5)], a[(14,5,4)], a[(14,5,3)], a[(14,5,2)], '', '4', a[(13,4,13)], a[(13,4,12)], a[(13,4,11)], a[(13,4,10)], a[(13,4,9)], a[(13,4,8)], a[(13,4,7)], a[(13,4,6)], a[(13,4,5)], a[(13,4,4)], a[(13,4,3)], a[(13,4,2)], '', '3', a[(12,3,12)], a[(12,3,11)], a[(12,3,10)], a[(12,3,9)], a[(12,3,8)], a[(12,3,7)], a[(12,3,6)], a[(12,3,5)], a[(12,3,4)], a[(12,3,3)], a[(12,3,2)], a[(12,3,1)], a[(12,3,0)]])
        csv_writer.writerow(['4', a[(14,4,14)], a[(14,4,13)], a[(14,4,12)], a[(14,4,11)], a[(14,4,10)], a[(14,4,9)], a[(14,4,8)], a[(14,4,7)], a[(14,4,6)], a[(14,4,5)], a[(14,4,4)], a[(14,4,3)], a[(14,4,2)], '', '3', a[(13,3,13)], a[(13,3,12)], a[(13,3,11)], a[(13,3,10)], a[(13,3,9)], a[(13,3,8)], a[(13,3,7)], a[(13,3,6)], a[(13,3,5)], a[(13,3,4)], a[(13,3,3)], a[(13,3,2)], '', '2', a[(12,2,12)], a[(12,2,11)], a[(12,2,10)], a[(12,2,9)], a[(12,2,8)], a[(12,2,7)], a[(12,2,6)], a[(12,2,5)], a[(12,2,4)], a[(12,2,3)], a[(12,2,2)], a[(12,2,1)], a[(12,2,0)]])
        csv_writer.writerow(['3', a[(14,3,14)], a[(14,3,13)], a[(14,3,12)], a[(14,3,11)], a[(14,3,10)], a[(14,3,9)], a[(14,3,8)], a[(14,3,7)], a[(14,3,6)], a[(14,3,5)], a[(14,3,4)], a[(14,3,3)], a[(14,3,2)], '', '2', a[(13,2,13)], a[(13,2,12)], a[(13,2,11)], a[(13,2,10)], a[(13,2,9)], a[(13,2,8)], a[(13,2,7)], a[(13,2,6)], a[(13,2,5)], a[(13,2,4)], a[(13,2,3)], a[(13,2,2)], '', '', a[(12,1,12)], a[(12,1,11)], a[(12,1,10)], a[(12,1,9)], a[(12,1,8)], a[(12,1,7)], a[(12,1,6)], a[(12,1,5)], a[(12,1,4)], a[(12,1,3)], a[(12,1,2)], a[(12,1,1)], a[(12,1,0)]])
        csv_writer.writerow(['2', a[(14,2,14)], a[(14,2,13)], a[(14,2,12)], a[(14,2,11)], a[(14,2,10)], a[(14,2,9)], a[(14,2,8)], a[(14,2,7)], a[(14,2,6)], a[(14,2,5)], a[(14,2,4)], a[(14,2,3)], a[(14,2,2)], '', '', a[(13,1,13)], a[(13,1,12)], a[(13,1,11)], a[(13,1,10)], a[(13,1,9)], a[(13,1,8)], a[(13,1,7)], a[(13,1,6)], a[(13,1,5)], a[(13,1,4)], a[(13,1,3)], a[(13,1,2)], '', '', a[(12,0,12)], a[(12,0,11)], a[(12,0,10)], a[(12,0,9)], a[(12,0,8)], a[(12,0,7)], a[(12,0,6)], a[(12,0,5)], a[(12,0,4)], a[(12,0,3)], a[(12,0,2)], a[(12,0,1)], a[(12,0,0)]])
        csv_writer.writerow(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '','','','','','','','','','','','','','','','','','','','','','','','',''])    
        csv_writer.writerow(['', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2', '', '', 'T', '9', '8', '7', '6', '5', '4', '3', '2', '', '', '9', '8', '7', '6', '5', '4', '3', '2', '', '', '8', '7', '6', '5', '4', '3', '2'])    
        csv_writer.writerow(['J', a[(11,11,11)], a[(11,11,10)], a[(11,11,9)], a[(11,11,8)], a[(11,11,7)], a[(11,11,6)], a[(11,11,5)], a[(11,11,4)], a[(11,11,3)], a[(11,11,2)], '', 'T', a[(10,10,10)], a[(10,10,9)], a[(10,10,8)], a[(10,10,7)], a[(10,10,6)], a[(10,10,5)], a[(10,10,4)], a[(10,10,3)], a[(10,10,2)], '', '9', a[(9,9,9)], a[(9,9,8)], a[(9,9,7)], a[(9,9,6)], a[(9,9,5)], a[(9,9,4)], a[(9,9,3)], a[(9,9,2)], '', '8', a[(8,8,8)], a[(8,8,7)], a[(8,8,6)], a[(8,8,5)], a[(8,8,4)], a[(8,8,3)], a[(8,8,2)]])
        csv_writer.writerow(['T', a[(11,10,11)], a[(11,10,10)], a[(11,10,9)], a[(11,10,8)], a[(11,10,7)], a[(11,10,6)], a[(11,10,5)], a[(11,10,4)], a[(11,10,3)], a[(11,10,2)], '', '9', a[(10,9,10)], a[(10,9,9)], a[(10,9,8)], a[(10,9,7)], a[(10,9,6)], a[(10,9,5)], a[(10,9,4)], a[(10,9,3)], a[(10,9,2)], '', '8', a[(9,8,9)], a[(9,8,8)], a[(9,8,7)], a[(9,8,6)], a[(9,8,5)], a[(9,8,4)], a[(9,8,3)], a[(9,8,2)], '', '7', a[(8,7,8)], a[(8,7,7)], a[(8,7,6)], a[(8,7,5)], a[(8,7,4)], a[(8,7,3)], a[(8,7,2)]])
        csv_writer.writerow(['9', a[(11,9,11)], a[(11,9,10)], a[(11,9,9)], a[(11,9,8)], a[(11,9,7)], a[(11,9,6)], a[(11,9,5)], a[(11,9,4)], a[(11,9,3)], a[(11,9,2)], '', '8', a[(10,8,10)], a[(10,8,9)], a[(10,8,8)], a[(10,8,7)], a[(10,8,6)], a[(10,8,5)], a[(10,8,4)], a[(10,8,3)], a[(10,8,2)], '', '7', a[(9,7,9)], a[(9,7,8)], a[(9,7,7)], a[(9,7,6)], a[(9,7,5)], a[(9,7,4)], a[(9,7,3)], a[(9,7,2)], '', '6', a[(8,6,8)], a[(8,6,7)], a[(8,6,6)], a[(8,6,5)], a[(8,6,4)], a[(8,6,3)], a[(8,6,2)]])
        csv_writer.writerow(['8', a[(11,8,11)], a[(11,8,10)], a[(11,8,9)], a[(11,8,8)], a[(11,8,7)], a[(11,8,6)], a[(11,8,5)], a[(11,8,4)], a[(11,8,3)], a[(11,8,2)], '', '7', a[(10,7,10)], a[(10,7,9)], a[(10,7,8)], a[(10,7,7)], a[(10,7,6)], a[(10,7,5)], a[(10,7,4)], a[(10,7,3)], a[(10,7,2)], '', '6', a[(9,6,9)], a[(9,6,8)], a[(9,6,7)], a[(9,6,6)], a[(9,6,5)], a[(9,6,4)], a[(9,6,3)], a[(9,6,2)], '', '5', a[(8,5,8)], a[(8,5,7)], a[(8,5,6)], a[(8,5,5)], a[(8,5,4)], a[(8,5,3)], a[(8,5,2)]])
        csv_writer.writerow(['7', a[(11,7,11)], a[(11,7,10)], a[(11,7,9)], a[(11,7,8)], a[(11,7,7)], a[(11,7,6)], a[(11,7,5)], a[(11,7,4)], a[(11,7,3)], a[(11,7,2)], '', '6', a[(10,6,10)], a[(10,6,9)], a[(10,6,8)], a[(10,6,7)], a[(10,6,6)], a[(10,6,5)], a[(10,6,4)], a[(10,6,3)], a[(10,6,2)], '', '5', a[(9,5,9)], a[(9,5,8)], a[(9,5,7)], a[(9,5,6)], a[(9,5,5)], a[(9,5,4)], a[(9,5,3)], a[(9,5,2)], '', '4', a[(8,4,8)], a[(8,4,7)], a[(8,4,6)], a[(8,4,5)], a[(8,4,4)], a[(8,4,3)], a[(8,4,2)]])
        csv_writer.writerow(['6', a[(11,6,11)], a[(11,6,10)], a[(11,6,9)], a[(11,6,8)], a[(11,6,7)], a[(11,6,6)], a[(11,6,5)], a[(11,6,4)], a[(11,6,3)], a[(11,6,2)], '', '5', a[(10,5,10)], a[(10,5,9)], a[(10,5,8)], a[(10,5,7)], a[(10,5,6)], a[(10,5,5)], a[(10,5,4)], a[(10,5,3)], a[(10,5,2)], '', '4', a[(9,4,9)], a[(9,4,8)], a[(9,4,7)], a[(9,4,6)], a[(9,4,5)], a[(9,4,4)], a[(9,4,3)], a[(9,4,2)], '', '3', a[(8,3,8)], a[(8,3,7)], a[(8,3,6)], a[(8,3,5)], a[(8,3,4)], a[(8,3,3)], a[(8,3,2)]])
        csv_writer.writerow(['5', a[(11,5,11)], a[(11,5,10)], a[(11,5,9)], a[(11,5,8)], a[(11,5,7)], a[(11,5,6)], a[(11,5,5)], a[(11,5,4)], a[(11,5,3)], a[(11,5,2)], '', '4', a[(10,4,10)], a[(10,4,9)], a[(10,4,8)], a[(10,4,7)], a[(10,4,6)], a[(10,4,5)], a[(10,4,4)], a[(10,4,3)], a[(10,4,2)], '', '3', a[(9,3,9)], a[(9,3,8)], a[(9,3,7)], a[(9,3,6)], a[(9,3,5)], a[(9,3,4)], a[(9,3,3)], a[(9,3,2)], '', '2', a[(8,2,8)], a[(8,2,7)], a[(8,2,6)], a[(8,2,5)], a[(8,2,4)], a[(8,2,3)], a[(8,2,2)]])
        csv_writer.writerow(['4', a[(11,4,11)], a[(11,4,10)], a[(11,4,9)], a[(11,4,8)], a[(11,4,7)], a[(11,4,6)], a[(11,4,5)], a[(11,4,4)], a[(11,4,3)], a[(11,4,2)], '', '3', a[(10,3,10)], a[(10,3,9)], a[(10,3,8)], a[(10,3,7)], a[(10,3,6)], a[(10,3,5)], a[(10,3,4)], a[(10,3,3)], a[(10,3,2)], '', '2', a[(9,2,9)], a[(9,2,8)], a[(9,2,7)], a[(9,2,6)], a[(9,2,5)], a[(9,2,4)], a[(9,2,3)], a[(9,2,2)], '', '', a[(8,1,8)], a[(8,1,7)], a[(8,1,6)], a[(8,1,5)], a[(8,1,4)], a[(8,1,3)], a[(8,1,2)]])
        csv_writer.writerow(['3', a[(11,3,11)], a[(11,3,10)], a[(11,3,9)], a[(11,3,8)], a[(11,3,7)], a[(11,3,6)], a[(11,3,5)], a[(11,3,4)], a[(11,3,3)], a[(11,3,2)], '', '2', a[(10,2,10)], a[(10,2,9)], a[(10,2,8)], a[(10,2,7)], a[(10,2,6)], a[(10,2,5)], a[(10,2,4)], a[(10,2,3)], a[(10,2,2)], '', '', a[(9,1,9)], a[(9,1,8)], a[(9,1,7)], a[(9,1,6)], a[(9,1,5)], a[(9,1,4)], a[(9,1,3)], a[(9,1,2)], '', '', a[(8,0,8)], a[(8,0,7)], a[(8,0,6)], a[(8,0,5)], a[(8,0,4)], a[(8,0,3)], a[(8,0,2)]])
        csv_writer.writerow(['2', a[(11,2,11)], a[(11,2,10)], a[(11,2,9)], a[(11,2,8)], a[(11,2,7)], a[(11,2,6)], a[(11,2,5)], a[(11,2,4)], a[(11,2,3)], a[(11,2,2)], '', '', a[(10,1,10)], a[(10,1,9)], a[(10,1,8)], a[(10,1,7)], a[(10,1,6)], a[(10,1,5)], a[(10,1,4)], a[(10,1,3)], a[(10,1,2)], '', '', a[(9,0,9)], a[(9,0,8)], a[(9,0,7)], a[(9,0,6)], a[(9,0,5)], a[(9,0,4)], a[(9,0,3)], a[(9,0,2)], '', '', a[(8,-1,8)], a[(8,-1,7)], a[(8,-1,6)], a[(8,-1,5)], a[(8,-1,4)], a[(8,-1,3)], a[(8,-1,2)]])
        csv_writer.writerow(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '','','','','','','','','','','','','','','','','','','','','','','','',''])    
        csv_writer.writerow(['', '7', '6', '5', '4', '3', '2', '', '', '6', '5', '4', '3', '2', '', '', '5', '4', '3', '2', '', '', '4', '3', '2', '', '', '3', '2', '', '', '2'])
        csv_writer.writerow(['7', a[(7,7,7)], a[(7,7,6)], a[(7,7,5)], a[(7,7,4)], a[(7,7,3)], a[(7,7,2)], '', '6', a[(6,6,6)], a[(6,6,5)], a[(6,6,4)], a[(6,6,3)], a[(6,6,2)], '', '5', a[(5,5,5)], a[(5,5,4)], a[(5,5,3)], a[(5,5,2)], '', '4', a[(4,4,4)], a[(4,4,3)], a[(4,4,2)], '', '3', a[(3,3,3)], a[(3,3,2)], '', '2', a[(2,2,2)]])
        csv_writer.writerow(['6', a[(7,6,7)], a[(7,6,6)], a[(7,6,5)], a[(7,6,4)], a[(7,6,3)], a[(7,6,2)], '', '5', a[(6,5,6)], a[(6,5,5)], a[(6,5,4)], a[(6,5,3)], a[(6,5,2)], '', '4', a[(5,4,5)], a[(5,4,4)], a[(5,4,3)], a[(5,4,2)], '', '3', a[(4,3,4)], a[(4,3,3)], a[(4,3,2)], '', '2', a[(3,2,3)], a[(3,2,2)], '', '', a[(2,1,2)]])
        csv_writer.writerow(['5', a[(7,5,7)], a[(7,5,6)], a[(7,5,5)], a[(7,5,4)], a[(7,5,3)], a[(7,5,2)], '', '4', a[(6,4,6)], a[(6,4,5)], a[(6,4,4)], a[(6,4,3)], a[(6,4,2)], '', '3', a[(5,3,5)], a[(5,3,4)], a[(5,3,3)], a[(5,3,2)], '', '2', a[(4,2,4)], a[(4,2,3)], a[(4,2,2)], '', '', a[(3,1,3)], a[(3,1,2)], '', '', a[(2,0,2)]])
        csv_writer.writerow(['4', a[(7,4,7)], a[(7,4,6)], a[(7,4,5)], a[(7,4,4)], a[(7,4,3)], a[(7,4,2)], '', '3', a[(6,3,6)], a[(6,3,5)], a[(6,3,4)], a[(6,3,3)], a[(6,3,2)], '', '2', a[(5,2,5)], a[(5,2,4)], a[(5,2,3)], a[(5,2,2)], '', '', a[(4,1,4)], a[(4,1,3)], a[(4,1,2)], '', '', a[(3,0,3)], a[(3,0,2)], '', '', a[(2,-1,2)]])
        csv_writer.writerow(['3', a[(7,3,7)], a[(7,3,6)], a[(7,3,5)], a[(7,3,4)], a[(7,3,3)], a[(7,3,2)], '', '2', a[(6,2,6)], a[(6,2,5)], a[(6,2,4)], a[(6,2,3)], a[(6,2,2)], '', '', a[(5,1,5)], a[(5,1,4)], a[(5,1,3)], a[(5,1,2)], '', '', a[(4,0,4)], a[(4,0,3)], a[(4,0,2)], '', '', a[(3,-1,3)], a[(3,-1,2)], '', '', a[(2,-2,2)]])
        csv_writer.writerow(['2', a[(7,2,7)], a[(7,2,6)], a[(7,2,5)], a[(7,2,4)], a[(7,2,3)], a[(7,2,2)], '', '', a[(6,1,6)], a[(6,1,5)], a[(6,1,4)], a[(6,1,3)], a[(6,1,2)], '', '', a[(5,0,5)], a[(5,0,4)], a[(5,0,3)], a[(5,0,2)], '', '', a[(4,-1,4)], a[(4,-1,3)], a[(4,-1,2)], '', '', a[(3,-2,3)], a[(3,-2,2)], '', '', a[(2,-3,2)]])






FileNotFoundError: File b'../reports/condensed_results/two-tone/55 SB 3Bet vs BN.csv' does not exist

In [5]:
from pyexcel.cookbook import merge_all_to_a_book
# import pyexcel.ext.xlsx # no longer required if you use pyexcel >= 0.2.2 
import glob


merge_all_to_a_book(glob.glob("../reports/final_results/{}/*.csv".format(board_type)), "../reports/final_results/{}.xlsx".format(board_type))

## Old Code

In [10]:
m2 = {14: 'A', 13: 'K', 12: 'Q', 11: 'J', 10: 'T',
                  9: '9', 8: '8', 7: '7', 6: '6', 5: '5',
                  4: '4', 3: '3', 2: '2', 1: '', 0: '', -1: '', 
                  -2: '', -3: '', -4: '', -5: '', -6: '', -7: '', 
                  -8: '', -9: '', -10: '', -11: '', -12: '', -13: '', }


for row in range(7,1,-1): 
    pre_string = ["    csv_writer.writerow(["]
    string1 = ["'" + str(m2[row]) + "'"]
    string2 = ["'" + str(m2[row-1]) + "'"]
    string3 = ["'" + str(m2[row-2]) + "'"]
    string4 = ["'" + str(m2[row-3]) + "'"]
    string5 = ["'" + str(m2[row-4]) + "'"]
    string6 = ["'" + str(m2[row-5]) + "'"]
    for column in range(7,1,-1):
        string1.append("a[(7,{},{})]".format(row, column))
        if column-1 > 1:
            string2.append("a[(6,{},{})]".format(row-1, column-1))
        if column-2 > 1:
            string3.append("a[(5,{},{})]".format(row-2, column-2))
        if column-3 > 1:
            string4.append("a[(4,{},{})]".format(row-3, column-3))
        if column-4 > 1:
            string5.append("a[(3,{},{})]".format(row-4, column-4))
        if column-5 > 1:
            string6.append("a[(2,{},{})]".format(row-5, column-5))            
    print("".join(pre_string) + ", ".join(string1) + ", '', " + ", ".join(string2) + ", '', " + ", ".join(string3) + ", '', " + ", ".join(string4) + ", '', " + ", ".join(string5) + ", '', " + ", ".join(string6) + "])")
    

    csv_writer.writerow(['7', a[(7,7,7)], a[(7,7,6)], a[(7,7,5)], a[(7,7,4)], a[(7,7,3)], a[(7,7,2)], '', '6', a[(6,6,6)], a[(6,6,5)], a[(6,6,4)], a[(6,6,3)], a[(6,6,2)], '', '5', a[(5,5,5)], a[(5,5,4)], a[(5,5,3)], a[(5,5,2)], '', '4', a[(4,4,4)], a[(4,4,3)], a[(4,4,2)], '', '3', a[(3,3,3)], a[(3,3,2)], '', '2', a[(2,2,2)]])
    csv_writer.writerow(['6', a[(7,6,7)], a[(7,6,6)], a[(7,6,5)], a[(7,6,4)], a[(7,6,3)], a[(7,6,2)], '', '5', a[(6,5,6)], a[(6,5,5)], a[(6,5,4)], a[(6,5,3)], a[(6,5,2)], '', '4', a[(5,4,5)], a[(5,4,4)], a[(5,4,3)], a[(5,4,2)], '', '3', a[(4,3,4)], a[(4,3,3)], a[(4,3,2)], '', '2', a[(3,2,3)], a[(3,2,2)], '', '', a[(2,1,2)]])
    csv_writer.writerow(['5', a[(7,5,7)], a[(7,5,6)], a[(7,5,5)], a[(7,5,4)], a[(7,5,3)], a[(7,5,2)], '', '4', a[(6,4,6)], a[(6,4,5)], a[(6,4,4)], a[(6,4,3)], a[(6,4,2)], '', '3', a[(5,3,5)], a[(5,3,4)], a[(5,3,3)], a[(5,3,2)], '', '2', a[(4,2,4)], a[(4,2,3)], a[(4,2,2)], '', '', a[(3,1,3)], a[(3,1,2)], '', '', a[(2,0,2)]])
    csv_writer.writ

In [88]:
m2 = {14: 'A', 13: 'K', 12: 'Q', 11: 'J', 10: 'T',
                  9: '9', 8: '8', 7: '7', 6: '6', 5: '5',
                  4: '4', 3: '3', 2: '2', 1: '', 0: '', -1: '', 
                  -2: '', -3: '', -4: '', -5: '', -6: '', -7: '', 
                  -8: '', -9: '', -10: '', -11: '', -12: '', -13: '', }


for row in range(11,1,-1): 
    pre_string = ["    csv_writer.writerow(["]
    string1 = ["'" + str(m2[row]) + "'"]
    string2 = ["'" + str(m2[row-1]) + "'"]
    string3 = ["'" + str(m2[row-2]) + "'"]
    string4 = ["'" + str(m2[row-3]) + "'"]
    for column in range(11,1,-1):
        string1.append("a[(11,{},{})]".format(row, column))
        if column-1 > 1:
            string2.append("a[(10,{},{})]".format(row-1, column-1))
        if column-2 > 1:
            string3.append("a[(9,{},{})]".format(row-2, column-2))
        if column-3 > 1:
            string4.append("a[(8,{},{})]".format(row-3, column-3))
    print("".join(pre_string) + ", ".join(string1) + ", '', " + ", ".join(string2) + ", '', " + ", ".join(string3) + ", '', " + ", ".join(string4) + "])")
    

    csv_writer.writerow(['J', a[(11,11,11)], a[(11,11,10)], a[(11,11,9)], a[(11,11,8)], a[(11,11,7)], a[(11,11,6)], a[(11,11,5)], a[(11,11,4)], a[(11,11,3)], a[(11,11,2)], '', 'T', a[(10,10,10)], a[(10,10,9)], a[(10,10,8)], a[(10,10,7)], a[(10,10,6)], a[(10,10,5)], a[(10,10,4)], a[(10,10,3)], a[(10,10,2)], '', '9', a[(9,9,9)], a[(9,9,8)], a[(9,9,7)], a[(9,9,6)], a[(9,9,5)], a[(9,9,4)], a[(9,9,3)], a[(9,9,2)], '', '8', a[(8,8,8)], a[(8,8,7)], a[(8,8,6)], a[(8,8,5)], a[(8,8,4)], a[(8,8,3)], a[(8,8,2)]])
    csv_writer.writerow(['T', a[(11,10,11)], a[(11,10,10)], a[(11,10,9)], a[(11,10,8)], a[(11,10,7)], a[(11,10,6)], a[(11,10,5)], a[(11,10,4)], a[(11,10,3)], a[(11,10,2)], '', '9', a[(10,9,10)], a[(10,9,9)], a[(10,9,8)], a[(10,9,7)], a[(10,9,6)], a[(10,9,5)], a[(10,9,4)], a[(10,9,3)], a[(10,9,2)], '', '8', a[(9,8,9)], a[(9,8,8)], a[(9,8,7)], a[(9,8,6)], a[(9,8,5)], a[(9,8,4)], a[(9,8,3)], a[(9,8,2)], '', '7', a[(8,7,8)], a[(8,7,7)], a[(8,7,6)], a[(8,7,5)], a[(8,7,4)], a[(8,7,3)], a[(8,7,2

In [86]:
m2 = {14: 'A', 13: 'K', 12: 'Q', 11: 'J', 10: 'T',
                  9: '9', 8: '8', 7: '7', 6: '6', 5: '5',
                  4: '4', 3: '3', 2: '2', 1: '', 0: ''}


for row in range(14,1,-1): 
    pre_string = ["    csv_writer.writerow(["]
    string1 = ["'" + str(m2[row]) + "'"]
    string2 = ["'" + str(m2[row-1]) + "'"]
    string3 = ["'" + str(m2[row-2]) + "'"]
    for column in range(14,1,-1):
        string1.append("a[(14,{},{})]".format(row, column))
        if column-1 > 1:
            string2.append("a[(13,{},{})]".format(row-1, column-1))
        string3.append("a[(12,{},{})]".format(row-2, column-2))
    print("".join(pre_string) + ", ".join(string1) + ", '', " + ", ".join(string2) + ", '', " + ", ".join(string3) + "])")
    

    csv_writer.writerow(['A', a[(14,14,14)], a[(14,14,13)], a[(14,14,12)], a[(14,14,11)], a[(14,14,10)], a[(14,14,9)], a[(14,14,8)], a[(14,14,7)], a[(14,14,6)], a[(14,14,5)], a[(14,14,4)], a[(14,14,3)], a[(14,14,2)], '', 'K', a[(13,13,13)], a[(13,13,12)], a[(13,13,11)], a[(13,13,10)], a[(13,13,9)], a[(13,13,8)], a[(13,13,7)], a[(13,13,6)], a[(13,13,5)], a[(13,13,4)], a[(13,13,3)], a[(13,13,2)], '', 'Q', a[(12,12,12)], a[(12,12,11)], a[(12,12,10)], a[(12,12,9)], a[(12,12,8)], a[(12,12,7)], a[(12,12,6)], a[(12,12,5)], a[(12,12,4)], a[(12,12,3)], a[(12,12,2)], a[(12,12,1)], a[(12,12,0)]])
    csv_writer.writerow(['K', a[(14,13,14)], a[(14,13,13)], a[(14,13,12)], a[(14,13,11)], a[(14,13,10)], a[(14,13,9)], a[(14,13,8)], a[(14,13,7)], a[(14,13,6)], a[(14,13,5)], a[(14,13,4)], a[(14,13,3)], a[(14,13,2)], '', 'Q', a[(13,12,13)], a[(13,12,12)], a[(13,12,11)], a[(13,12,10)], a[(13,12,9)], a[(13,12,8)], a[(13,12,7)], a[(13,12,6)], a[(13,12,5)], a[(13,12,4)], a[(13,12,3)], a[(13,12,2)], '', 'J', 